In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Property_taxes_data/property_tax_report_csv2019.csv')

neigh_codes = pd.read_csv('van_neighbour_code_map.csv')

In [2]:
data_n = pd.merge(data, neigh_codes, left_on='NEIGHBOURHOOD_CODE', right_on='CODE', how='left')
data_n.head()

,PID,LEGAL_TYPE,FOLIO,LAND_COORDINATE,ZONE_NAME,ZONE_CATEGORY,LOT,BLOCK,PLAN,DISTRICT_LOT,...,CURRENT_IMPROVEMENT_VALUE,TAX_ASSESSMENT_YEAR,PREVIOUS_LAND_VALUE,PREVIOUS_IMPROVEMENT_VALUE,YEAR_BUILT,BIG_IMPROVEMENT_YEAR,TAX_LEVY,NEIGHBOURHOOD_CODE,CODE,NEIGHBOURHOOD_NAME
0,029-080-100,STRATA,638179040086,63817904,CD-1 (462),Comprehensive Development,86,NaN,EPS1197,200A,...,186000.0,2019.0,426000.0,187000.0,2013.0,2013.0,NaN,13,13,Mount Pleasant
1,018-056-971,LAND,230600060000,23060006,I-2,Light Industrial,1,NaN,LMP7979,182,...,19000.0,2019.0,6277000.0,19000.0,1993.0,1993.0,NaN,14,14,Grandview-Woodland
2,029-850-045,STRATA,138600890424,13860089,CD-1 (525),Comprehensive Development,424,54,EPS3242,541,...,NaN,NaN,2079000.0,644000.0,2016.0,2016.0,NaN,26,26,Downtown
3,009-272-046,LAND,150765720000,15076572,RS-1,One Family Dwelling,10,1008,VAP10874,526,...,57200.0,2019.0,3483000.0,57200.0,1965.0,1965.0,NaN,11,11,Oakridge
4,009-272-054,LAND,150765860000,15076586,RS-1,One Family Dwelling,11,1008,VAP10874,526,...,214000.0,2019.0,3210000.0,214000.0,1973.0,1973.0,NaN,11,11,Oakridge


In [3]:
data_n['CURRENT_HOUSE_PRICE'] = data_n.apply(lambda x: (x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE'])/1000000.0, axis = 1)
data_n['PREVIOUS_HOUSE_PRICE'] = data_n.apply(lambda x: (x['PREVIOUS_LAND_VALUE']+x['PREVIOUS_IMPROVEMENT_VALUE'])/1000000.0, axis = 1)
data_n['PRICE_CHANGE_FROM_PREV_YEAR'] = data_n['CURRENT_HOUSE_PRICE']-data_n['PREVIOUS_HOUSE_PRICE']
data_n.columns = data_n.columns.str.strip()

In [4]:
req_data = data_n[['NEIGHBOURHOOD_NAME','LEGAL_TYPE','CURRENT_HOUSE_PRICE','PREVIOUS_HOUSE_PRICE','YEAR_BUILT']]

In [5]:
val_counts = req_data['NEIGHBOURHOOD_NAME'].value_counts()
neighbour_name = val_counts.rename_axis('Neighbourhood_Name').reset_index(name='Property_Count')

In [6]:
land_data = req_data[req_data['LEGAL_TYPE']=='LAND']
val_counts = land_data['NEIGHBOURHOOD_NAME'].value_counts()
Land_vals = val_counts.rename_axis('Neighbourhood_Name').reset_index(name='Land_Property_Count')

In [7]:
strata_data = req_data[req_data['LEGAL_TYPE']=='STRATA']
val_counts = strata_data['NEIGHBOURHOOD_NAME'].value_counts()
Strata_vals = val_counts.rename_axis('Neighbourhood_Name').reset_index(name='Strata_Property_Count')

In [8]:
Price_vals = req_data.groupby('NEIGHBOURHOOD_NAME').agg({'CURRENT_HOUSE_PRICE':np.median}).reset_index()
Price_vals = Price_vals.rename(columns={"NEIGHBOURHOOD_NAME": "Neighbourhood_Name", "CURRENT_HOUSE_PRICE": "Median_House_Price_2019"})

In [9]:
req_data['Price_Change'] = req_data['CURRENT_HOUSE_PRICE']-req_data['PREVIOUS_HOUSE_PRICE']
Price_decrease = req_data[req_data['Price_Change']<=0]
val_counts = Price_decrease['NEIGHBOURHOOD_NAME'].value_counts()
inc_counts = val_counts.rename_axis('Neighbourhood_Name').reset_index(name='Price_decrease_Count')
prop_counts = pd.merge(neighbour_name, inc_counts, on = 'Neighbourhood_Name', how='inner')
prop_counts['Propertes_Price_decrease_percent_from_2018'] = (prop_counts['Price_decrease_Count']/prop_counts['Property_Count'])*100
prop_counts = prop_counts[['Neighbourhood_Name','Propertes_Price_decrease_percent_from_2018']]

c:\users\rohithsoorampc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
req_data = req_data.dropna()
req_data['AGE'] = 2019 - req_data['YEAR_BUILT'].astype(int) 

Prop_age = req_data.groupby('NEIGHBOURHOOD_NAME').agg({'AGE':np.median}).reset_index()
Prop_age = Prop_age.rename(columns={"NEIGHBOURHOOD_NAME": "Neighbourhood_Name", "AGE": "Median_House_Age"})

In [11]:
#from functools import reduce
df_list = [neighbour_name, Land_vals, Strata_vals, Price_vals,prop_counts,Prop_age]
fin_df = df_list[0]
for df in df_list[1:]:
    fin_df = fin_df.merge(df, on='Neighbourhood_Name')
fin_df.to_csv("Vancouver_Neigbourhood_Data.csv",index=False)